In [ ]:
# | default_exp _helpers.utils

In [ ]:
# | export


import os
from configparser import ConfigParser
from contextlib import contextmanager
from pathlib import Path
from typing import *
from urllib.parse import urlparse

import nbdev
import typer

In [ ]:
import shutil
from tempfile import TemporaryDirectory

import numpy as np
import pytest

In [ ]:
# | export


@contextmanager
def set_cwd(cwd_path: Union[Path, str]) -> Generator:
    """Set the current working directory for the duration of the context manager.

    Args:
        cwd_path: The path to the new working directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)

    try:
        nbdev.config.get_config.cache_clear()
        yield
    finally:
        os.chdir(original_cwd)

In [ ]:
with TemporaryDirectory() as d:
    with set_cwd(d):
        assert (
            Path(os.getcwd()) == Path(d).resolve()
        ), f"{os.getcwd()}, {Path(d).resolve()}"

In [ ]:
# | export


def get_value_from_config(root_path: str, config_name: str) -> str:
    """Get a value from the configuration file.

    Args:
        root_path: The root path of the configuration file.
        config_name: The name of the configuration to get.

    Returns:
        The value of the configuration.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    if not config.has_option("DEFAULT", config_name):
        return ""
    return config["DEFAULT"][config_name]

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    ret_val = get_value_from_config(d, "lib_path")
    print(ret_val)
    assert ret_val == "nbdev_mkdocs", ret_val

    ret_val = get_value_from_config(d, "repo")
    print(ret_val)
    assert ret_val == "nbdev-mkdocs", ret_val

    ret_val = get_value_from_config(d, "user")
    print(ret_val)
    assert ret_val == "airtai", ret_val

nbdev_mkdocs
nbdev-mkdocs
airtai


In [ ]:
# | export


def is_local_path(path: str) -> bool:
    # Check if the path is an absolute path
    """Check if a path is a local path.

    Args:
        path: The path to check.

    Returns:
        True if the path is a local path, False otherwise.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    if os.path.isabs(path):
        return True

    # Check if the path is a URL with a scheme (e.g. http, https, ftp)
    parsed_url = urlparse(path)
    if parsed_url.scheme:
        return False

    # If the path is not an absolute path and does not have a URL scheme,
    # it is assumed to be a local path
    return True

In [ ]:
assert is_local_path("/tmp/abc/file.txt")
assert not is_local_path("http://www.example.com")

In [ ]:
# | export


def add_counter_suffix_to_filename(src_path: Path) -> None:
    """Add a counter suffix to the given file

    Args:
        src_path: Path to the file to be renamed

    Returns:
        None

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    parent_dir = src_path.parent
    counter_suffix = (
        max(
            [
                int(f.stem.split(".")[1])
                for f in parent_dir.glob(f"{src_path.stem}.*.*")
            ],
            default=0,
        )
        + 1
    )
    dst_path = parent_dir / f"{src_path.stem}.{counter_suffix}{src_path.suffix}"
    os.rename(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    file_path = Path(d) / "social_image.txt"
    with open(file_path, "w") as f:
        f.write("sample_text")

    add_counter_suffix_to_filename(file_path)

    actual = [f.name for f in Path(d).glob(f"{file_path.stem}*.*")]
    print(actual)

    expected = ["social_image.1.txt"]
    np.testing.assert_array_equal(actual, expected)

    assert not file_path.exists()
    assert (Path(d) / "social_image.1.txt").exists()

['social_image.1.txt']


In [ ]:
with TemporaryDirectory() as d:
    for i in [
        "social_image.txt",
        "social_image.1.txt",
        "social_image.2.txt",
        "social_image.11.txt",
    ]:
        file_path = Path(d) / i
        with open(file_path, "w") as f:
            f.write("sample_text")

    file_path = Path(d) / "social_image.txt"
    add_counter_suffix_to_filename(file_path)

    actual = sorted([f.name for f in Path(d).glob(f"{file_path.stem}*.*")])
    print(actual)
    expected = sorted(
        [
            "social_image.1.txt",
            "social_image.2.txt",
            "social_image.11.txt",
            "social_image.12.txt",
        ]
    )
    np.testing.assert_array_equal(actual, expected)

    assert not file_path.exists()
    assert (Path(d) / "social_image.12.txt").exists()

['social_image.1.txt', 'social_image.11.txt', 'social_image.12.txt', 'social_image.2.txt']


In [ ]:
# | export

def raise_error_and_exit(message: str) -> None:
    """Raise an error and exit

    Args:
        message: The error message to display

    Returns:
        None

    Raises:
        typer.Exit: If the error message is not provided

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    typer.secho(
        message,
        err=True,
        fg=typer.colors.RED,
    )
    raise typer.Exit(code=1)

In [ ]:
with pytest.raises(typer.Exit) as e:
    raise_error_and_exit("testing!!!")

testing!!!
